<a href="https://colab.research.google.com/github/tmoura/softexIA/blob/main/lstm_serie_temporal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM com série temporal

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import tensorflow as tf
import sklearn

In [2]:
np.__version__, pd.__version__, matplotlib.__version__, tf.__version__, sklearn.__version__

('1.26.4', '2.2.2', '3.8.0', '2.17.0', '1.5.2')

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [4]:
urlTrain = 'https://raw.githubusercontent.com/tmoura/machinelearning/refs/heads/master/datasets/petr4_treinamento.csv'
urlTest =  'https://raw.githubusercontent.com/tmoura/machinelearning/refs/heads/master/datasets/petr4_teste.csv'

base = pd.read_csv(urlTrain)
base

,Date,Open,High,Low,Close,Adj Close,Volume
0,2013-01-02,19.990000,20.209999,19.690001,19.690001,18.086271,30182600.0
1,2013-01-03,19.809999,20.400000,19.700001,20.400000,18.738441,30552600.0
2,2013-01-04,20.330000,20.620001,20.170000,20.430000,18.766001,36141000.0
3,2013-01-07,20.480000,20.670000,19.950001,20.080000,18.444506,28069600.0
4,2013-01-08,20.110001,20.230000,19.459999,19.500000,17.911745,29091300.0
...,...,...,...,...,...,...,...
1240,2017-12-25,15.750000,15.750000,15.750000,15.750000,15.718563,0.0
1241,2017-12-26,15.750000,15.990000,15.690000,15.970000,15.938125,22173100.0
1242,2017-12-27,15.990000,16.139999,15.980000,16.049999,16.017963,23552200.0
1243,2017-12-28,16.100000,16.129999,16.000000,16.100000,16.067865,19011500.0


In [5]:
base.isnull().sum()

,0
Date,0
Open,3
High,3
Low,3
Close,3
Adj Close,3
Volume,3


In [6]:
base = base.dropna()

In [7]:
base.shape

(1242, 7)

In [8]:
base_treinamento = base.iloc[:, 1:2].values

In [9]:
base_treinamento

array([[19.99    ],
       [19.809999],
       [20.33    ],
       ...,
       [15.99    ],
       [16.1     ],
       [16.1     ]])

In [10]:
normalizador = MinMaxScaler(feature_range=(0, 1))
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)
base_treinamento_normalizada

array([[0.76501938],
       [0.7562984 ],
       [0.78149225],
       ...,
       [0.57122093],
       [0.57655039],
       [0.57655039]])

In [11]:
base_treinamento_normalizada.shape

(1242, 1)

In [12]:
X = [] # previsores
y = [] # preço real
for i in range(90, 1242): # 90 preços anteriores
  X.append(base_treinamento_normalizada[i - 90:i, 0])
  y.append(base_treinamento_normalizada[i, 0])
  #print(i, i - 90)
X, y = np.array(X), np.array(y)

In [13]:
X.shape

(1152, 90)

In [14]:
y.shape

(1152,)

In [15]:
X[0], y[0]

(array([0.76501938, 0.7562984 , 0.78149225, 0.78875969, 0.77083338,
        0.74806197, 0.75436047, 0.75823643, 0.76598837, 0.76598837,
        0.76017437, 0.75872098, 0.75581391, 0.74467054, 0.7374031 ,
        0.7374031 , 0.73498067, 0.75242248, 0.73401163, 0.71656977,
        0.68120155, 0.67538755, 0.67635659, 0.63372098, 0.66521318,
        0.65649225, 0.64680228, 0.66618222, 0.65843028, 0.64970935,
        0.65116274, 0.66424419, 0.67344961, 0.64292631, 0.64486434,
        0.62257747, 0.60949617, 0.60998067, 0.60852713, 0.59593023,
        0.61143411, 0.60222863, 0.64922481, 0.68362398, 0.70687989,
        0.68265509, 0.70978682, 0.70784879, 0.71608527, 0.73643411,
        0.7122093 , 0.7122093 , 0.7194767 , 0.70348832, 0.69525189,
        0.70397287, 0.70397287, 0.69767442, 0.68168605, 0.68168605,
        0.65310078, 0.66618222, 0.64825581, 0.66182175, 0.64341085,
        0.67877902, 0.69137592, 0.66569772, 0.65406982, 0.64292631,
        0.64147292, 0.63565891, 0.67587209, 0.68

In [16]:
base[0:90]

,Date,Open,High,Low,Close,Adj Close,Volume
0,2013-01-02,19.990000,20.209999,19.690001,19.690001,18.086271,30182600.0
1,2013-01-03,19.809999,20.400000,19.700001,20.400000,18.738441,30552600.0
2,2013-01-04,20.330000,20.620001,20.170000,20.430000,18.766001,36141000.0
3,2013-01-07,20.480000,20.670000,19.950001,20.080000,18.444506,28069600.0
4,2013-01-08,20.110001,20.230000,19.459999,19.500000,17.911745,29091300.0
...,...,...,...,...,...,...,...
87,2013-05-10,20.160000,20.389999,19.709999,19.959999,18.681934,22031200.0
88,2013-05-13,19.809999,19.850000,19.379999,19.490000,18.242029,21577600.0
89,2013-05-14,19.530001,19.680000,19.250000,19.549999,18.298189,30643700.0
90,2013-05-15,19.500000,19.639999,19.330000,19.570000,18.316908,21869300.0


In [17]:
X.shape, X.shape[1]

((1152, 90), 90)

In [18]:
regressor = Sequential()

# 100 unidades que mandam informação para ele mesmo (return_sequences=True)
regressor.add(LSTM(units = 100, return_sequences=True, input_shape = (X.shape[1], 1)))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 50, return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 50, return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.3))

regressor.add(Dense(units = 1, activation = 'linear'))

regressor.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 90, 100)             │          40,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 90, 100)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 90, 50)              │          30,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 90, 50)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 90, 50)              │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 90, 50)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 50)                  │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 111,451 (435.36 KB)

 Trainable params: 111,451 (435.36 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
regressor.compile(optimizer = 'rmsprop', loss = 'mean_squared_error', metrics = ['mean_absolute_error'])

In [ ]:
X.shape

In [20]:
regressor.fit(X, y, epochs = 100, batch_size = 32)

Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 20s 215ms/step - loss: 0.0703 - mean_absolute_error: 0.1922
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 6s 155ms/step - loss: 0.0145 - mean_absolute_error: 0.0941
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 11s 181ms/step - loss: 0.0130 - mean_absolute_error: 0.0902
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 11s 198ms/step - loss: 0.0118 - mean_absolute_error: 0.0851
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 9s 158ms/step - loss: 0.0111 - mean_absolute_error: 0.0807
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 191ms/step - loss: 0.0135 - mean_absolute_error: 0.0888
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 6s 153ms/step - loss: 0.0091 - mean_absolute_error: 0.0733
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 201ms/step - loss: 0.0073 - mean_absolute_error: 0.0649
Epoch 9/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 9s 172ms/step - loss: 0.0084 - mean_absolute_error: 0.0703
Epoch 10/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 169ms/step - loss: 0.0065 - mean_absolute_error: 0.0624
Epoch 11/100
36

In [22]:
base_teste = pd.read_csv(urlTest)
base_teste

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-01-02,16.190001,16.549999,16.190001,16.549999,16.516966,33461800
1,2018-01-03,16.490000,16.719999,16.370001,16.700001,16.666668,55940900
2,2018-01-04,16.780001,16.959999,16.620001,16.730000,16.696608,37064900
3,2018-01-05,16.700001,16.860001,16.570000,16.830000,16.796408,26958200
4,2018-01-08,16.740000,17.030001,16.709999,17.030001,16.996010,28400000
5,2018-01-09,17.030001,17.160000,16.959999,17.030001,16.996010,35070900
6,2018-01-10,16.920000,17.049999,16.770000,16.799999,16.766466,28547700
7,2018-01-11,16.879999,17.299999,16.840000,17.250000,17.215569,37921500
8,2018-01-12,17.040001,17.410000,17.020000,17.299999,17.265469,45912100
9,2018-01-15,17.320000,17.440001,17.150000,17.350000,17.315371,28945400


In [23]:
base_teste.shape

(22, 7)

In [24]:
y_teste = base_teste.iloc[:, 1:2].values

In [25]:
y_teste

array([[16.190001],
       [16.49    ],
       [16.780001],
       [16.700001],
       [16.74    ],
       [17.030001],
       [16.92    ],
       [16.879999],
       [17.040001],
       [17.32    ],
       [17.35    ],
       [17.92    ],
       [18.35    ],
       [18.309999],
       [18.26    ],
       [18.4     ],
       [18.42    ],
       [19.34    ],
       [19.620001],
       [19.67    ],
       [19.77    ],
       [19.74    ]])

In [26]:
base.shape

(1242, 7)

In [27]:
base_completa = pd.concat((base['Open'], base_teste['Open']), axis = 0)

In [28]:
1242 + 22

1264

In [29]:
base_completa.shape

(1264,)

In [30]:
base_completa

,Open
0,19.990000
1,19.809999
2,20.330000
3,20.480000
4,20.110001
...,...
17,19.340000
18,19.620001
19,19.670000
20,19.770000


In [31]:
len(base_completa), len(base_teste), 1264 - 22 - 90

(1264, 22, 1152)

In [ ]:
112 - 90

In [32]:
base_completa[1152:]

,Open
1155,13.930000
1156,13.760000
1157,13.790000
1158,13.530000
1159,13.850000
...,...
17,19.340000
18,19.620001
19,19.670000
20,19.770000


In [33]:
entradas = base_completa[len(base_completa) - len(base_teste) - 90:].values

In [35]:
entradas.shape

(112,)

In [34]:
entradas

array([13.93    , 13.76    , 13.79    , 13.53    , 13.85    , 13.96    ,
       14.57    , 14.65    , 15.02    , 15.1     , 14.88    , 14.98    ,
       14.94    , 15.03    , 15.07    , 15.02    , 15.1     , 15.25    ,
       15.85    , 15.6     , 15.79    , 15.86    , 15.7     , 15.37    ,
       15.5     , 15.19    , 15.6     , 15.9     , 15.88    , 15.66    ,
       15.61    , 16.129999, 16.17    , 16.08    , 16.23    , 16.16    ,
       16.139999, 16.219999, 16.      , 16.190001, 16.290001, 16.290001,
       16.530001, 16.780001, 16.77    , 16.969999, 16.9     , 16.99    ,
       16.9     , 16.959999, 17.049999, 17.309999, 16.690001, 16.889999,
       16.709999, 16.690001, 16.639999, 15.35    , 15.62    , 15.92    ,
       16.02    , 16.15    , 16.09    , 15.98    , 16.25    , 16.01    ,
       15.93    , 15.87    , 15.3     , 15.34    , 15.65    , 15.5     ,
       15.22    , 15.3     , 15.51    , 15.48    , 15.36    , 15.65    ,
       15.1     , 15.05    , 15.16    , 15.18    , 

In [36]:
entradas = entradas.reshape(-1, 1)
entradas.shape

(112, 1)

In [ ]:
entradas

In [37]:
entradas = normalizador.transform(entradas)

In [ ]:
entradas

In [45]:
X_teste = []
for i in range(90, 112):
  #print(i)
  X_teste.append(entradas[i - 90:i, 0])

In [48]:
X_teste = np.array(X_teste)

X_teste.shape

(22, 90)

In [49]:
previsoes = regressor.predict(X_teste)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 907ms/step


In [50]:
previsoes

array([[0.55832756],
       [0.56230557],
       [0.56651056],
       [0.57272863],
       [0.5795325 ],
       [0.58530784],
       [0.5908043 ],
       [0.5951501 ],
       [0.59747696],
       [0.5988025 ],
       [0.60143125],
       [0.6056115 ],
       [0.61359715],
       [0.62653625],
       [0.64110506],
       [0.6530834 ],
       [0.66111887],
       [0.66532487],
       [0.67199206],
       [0.68353534],
       [0.69748455],
       [0.71053326]], dtype=float32)

In [51]:
previsoes = normalizador.inverse_transform(previsoes)

In [ ]:
previsoes

In [ ]:
y_teste

In [53]:
previsoes.mean()

16.995205

In [54]:
y_teste.mean()

17.87454563636364

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_teste, previsoes)

In [ ]:
plt.plot(y_teste, color = "red", label = "Preço real")
plt.plot(previsoes, color = "blue", label = "Previsões")
plt.title("Previsão do preço das ações")
plt.xlabel("Tempo")
plt.ylabel("Valor")
plt.legend();